In [ ]:
import os
import cv2
import numpy as np
import sqlite3
from matplotlib import pyplot as plt
import ipywidgets as widgets
from IPython.display import display
import tempfile

# Thiết lập đường dẫn
DB_PATH = '../data/database.db'
IMG_OUTPUT_DIR = '../data/img/4goctrc/'
MODEL_DIR = '../models'
CONFIG_PATH = os.path.join(MODEL_DIR, 'yolo-obj.cfg')
WEIGHTS_PATH = os.path.join(MODEL_DIR, 'yolo-obj_final.weights')
NAMES_PATH = os.path.join(MODEL_DIR, 'obj.names')

# Tạo thư mục nếu chưa tồn tại
os.makedirs('../data', exist_ok=True)
os.makedirs(IMG_OUTPUT_DIR, exist_ok=True)

# Kết nối đến cơ sở dữ liệu
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

# Tạo bảng nếu chưa tồn tại
cursor.execute('''CREATE TABLE IF NOT EXISTS goctrc (input_name TEXT UNIQUE, output_path TEXT)''')
conn.commit()

# Hàm xóa ảnh cũ (nếu cần)
def clear_old_images():
    for file in os.listdir(IMG_OUTPUT_DIR):
        file_path = os.path.join(IMG_OUTPUT_DIR, file)
        if os.path.isfile(file_path):
            os.remove(file_path)
    print("Đã xóa các ảnh cũ trong thư mục đầu ra.")

# Tải các lớp từ file obj.names
classes = []
with open(NAMES_PATH, 'r') as f:
    classes = [line.strip() for line in f.readlines()]
print("Các lớp:", classes)

# Tải mô hình YOLO
net = cv2.dnn.readNet(WEIGHTS_PATH, CONFIG_PATH)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

CONF_THRESHOLD = 0.5
NMS_THRESHOLD = 0.4

# Hàm phát hiện đối tượng
def detect_objects(img_path):
    img = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    height, width, _ = img.shape

    blob = cv2.dnn.blobFromImage(img, 1/255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    outputs = net.forward(output_layers)

    class_ids, confidences, boxes = [], [], []

    for output in outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > CONF_THRESHOLD:
                center_x, center_y, w, h = (detection[0:4] * np.array([width, height, width, height])).astype('int')
                x, y = int(center_x - w / 2), int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, CONF_THRESHOLD, NMS_THRESHOLD)

    detections = []
    if len(indexes) > 0:
        for i in indexes.flatten():
            x, y, w, h = boxes[i]
            detections.append({
                'class_id': class_ids[i],
                'class_name': classes[class_ids[i]],
                'confidence': confidences[i],
                'box': [x, y, w, h]
            })

    return detections, img_rgb

# Hàm vẽ kết quả phát hiện
def draw_detections(img, detections):
    for det in detections:
        x, y, w, h = det['box']
        label = det['class_name']
        confidence = det['confidence']
        color = (255, 0, 0)  # Màu xanh dương cho bounding box

        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
        cv2.putText(img, f"{label} {confidence:.2f}", (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    plt.figure(figsize=(12, 8))
    plt.imshow(img)
    plt.axis('off')
    plt.show()

# Hàm lấy tọa độ các góc
def get_corner_points(detections):
    corners = {}
    for det in detections:
        class_name = det['class_name']
        box = det['box']
        x, y, w, h = box
        center_x = x + w / 2
        center_y = y + h / 2
        corners[class_name] = (center_x, center_y)
    return corners

# Hàm biến đổi phối cảnh
def four_point_transform(image, corners):
    required_keys = ['top-left', 'top-right', 'bottom-right', 'bottom-left']
    for key in required_keys:
        if key not in corners:
            raise ValueError(f"Thiếu góc: {key}")

    pts1 = np.float32([
        corners['top-left'],
        corners['top-right'],
        corners['bottom-right'],
        corners['bottom-left']
    ])

    widthA = np.linalg.norm(np.array(corners['bottom-right']) - np.array(corners['bottom-left']))
    widthB = np.linalg.norm(np.array(corners['top-right']) - np.array(corners['top-left']))
    maxWidth = max(int(widthA), int(widthB))

    heightA = np.linalg.norm(np.array(corners['top-right']) - np.array(corners['bottom-right']))
    heightB = np.linalg.norm(np.array(corners['top-left']) - np.array(corners['bottom-left']))
    maxHeight = max(int(heightA), int(heightB))

    pts2 = np.float32([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]
    ])

    matrix = cv2.getPerspectiveTransform(pts1, pts2)
    warped = cv2.warpPerspective(image, matrix, (maxWidth, maxHeight))

    return warped

# Hàm lưu và hiển thị ảnh (cập nhật để update thay vì insert mới)
def save_and_display(warped_image):
    input_image_name = uploaded_file_widget.value[0]['name']  
    
    output_image_path = os.path.join(IMG_OUTPUT_DIR,
                                      f"cropped_{os.path.basename(input_image_name)}")

    plt.figure(figsize=(12, 8))
    plt.imshow(cv2.cvtColor(warped_image, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title('Căn cước công dân đã cắt và chỉnh sửa')
    plt.show()

    # Kiểm tra xem ảnh đã tồn tại trong cơ sở dữ liệu chưa trước khi lưu
    cursor.execute('SELECT COUNT(*) FROM goctrc WHERE input_name=?', (input_image_name.split('/')[-1],))
    
    if cursor.fetchone()[0] == 0:  
        # Nếu không có bản ghi nào thì mới chèn vào cơ sở dữ liệu
        # Lưu ảnh đã cắt
        cv2.imwrite(output_image_path, warped_image)

        # Lưu đường dẫn vào cơ sở dữ liệu
        cursor.execute('INSERT INTO goctrc (input_name, output_path) VALUES (?, ?)', 
                       (input_image_name.split('/')[-1], output_image_path))
        
        print(f"Đã lưu ảnh mới vào cơ sở dữ liệu: {output_image_path}")
    
    else:
        # Nếu đã tồn tại thì có thể cập nhật đường dẫn mới hoặc thông báo
        cursor.execute('UPDATE goctrc SET output_path=? WHERE input_name=?', 
                       (output_image_path , input_image_name.split('/')[-1]))
        
        # Lưu ảnh đã cắt nếu cần thiết (có thể bỏ qua nếu không muốn ghi đè)
        cv2.imwrite(output_image_path , warped_image)  
        
        print(f"Đã cập nhật đường dẫn ảnh trong cơ sở dữ liệu: {output_image_path}")

def clear_and_display(warped_image):
   save_and_display(warped_image)

# Xóa dữ liệu cũ trước khi bắt đầu (có thể điều chỉnh tham số clear_data nếu cần)
clear_old_images()

# Tạo widget để chọn tệp ảnh
uploaded_file_widget = widgets.FileUpload(
    accept='image/*',
    multiple=False,
)

display(uploaded_file_widget)

def process_image(change):
    if uploaded_file_widget.value:
        # Lấy thông tin tệp được tải lên từ danh sách đầu tiên
        uploaded_file_info = uploaded_file_widget.value[0]  
        
        # Lưu tệp vào một vị trí tạm thời để xử lý
        with tempfile.NamedTemporaryFile(delete=False) as temp_file:
            temp_file.write(uploaded_file_info['content'])
            temp_file_path = temp_file.name
        
        # Thực hiện phát hiện và cắt góc
        detections , img_rgb= detect_objects(temp_file_path)
        
        draw_detections(img_rgb.copy(), detections)
        
        corners= get_corner_points(detections)

        try:
            warped_image= four_point_transform(cv2.cvtColor(img_rgb.copy(), cv2.COLOR_RGB2BGR), corners)
            clear_and_display(warped_image) 
            conn.commit()  
            
            print("Đã hoàn thành quá trình xử lý.")
            
        except ValueError as e:
            print(e)

uploaded_file_widget.observe(process_image) 

# Đóng kết nối cơ sở dữ liệu khi hoàn tất (nên thực hiện sau khi chắc chắn không còn thao tác nào nữa).
def close_connection():
   conn.close()


In [ ]:

# Thiết lập đường dẫn
DB_PATH = '../data/database.db'
IMG_OUTPUT_DIR = '../data/img/4gocsau/'
MODEL_DIR = '../models'
CONFIG_PATH = os.path.join(MODEL_DIR, 'yolo-obj3.cfg')
WEIGHTS_PATH = os.path.join(MODEL_DIR, 'yolo-obj3_final.weights')
NAMES_PATH = os.path.join(MODEL_DIR, 'obj3.names')

# Tạo thư mục nếu chưa tồn tại
os.makedirs('../data', exist_ok=True)
os.makedirs(IMG_OUTPUT_DIR, exist_ok=True)

# Kết nối đến cơ sở dữ liệu
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

# Tạo bảng nếu chưa tồn tại
cursor.execute('''CREATE TABLE IF NOT EXISTS gocsau (input_name TEXT UNIQUE, output_path TEXT)''')
conn.commit()

# Hàm xóa ảnh cũ (nếu cần)
def clear_old_images():
    for file in os.listdir(IMG_OUTPUT_DIR):
        file_path = os.path.join(IMG_OUTPUT_DIR, file)
        if os.path.isfile(file_path):
            os.remove(file_path)
    print("Đã xóa các ảnh cũ trong thư mục đầu ra.")

# Tải các lớp từ file obj.names
classes = []
with open(NAMES_PATH, 'r') as f:
    classes = [line.strip() for line in f.readlines()]
print("Các lớp:", classes)

# Tải mô hình YOLO
net = cv2.dnn.readNet(WEIGHTS_PATH, CONFIG_PATH)
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

CONF_THRESHOLD = 0.5
NMS_THRESHOLD = 0.4

# Hàm phát hiện đối tượng
def detect_objects(img_path):
    img = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    height, width, _ = img.shape

    blob = cv2.dnn.blobFromImage(img, 1/255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    outputs = net.forward(output_layers)

    class_ids, confidences, boxes = [], [], []

    for output in outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > CONF_THRESHOLD:
                center_x, center_y, w, h = (detection[0:4] * np.array([width, height, width, height])).astype('int')
                x, y = int(center_x - w / 2), int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, CONF_THRESHOLD, NMS_THRESHOLD)

    detections = []
    if len(indexes) > 0:
        for i in indexes.flatten():
            x, y, w, h = boxes[i]
            detections.append({
                'class_id': class_ids[i],
                'class_name': classes[class_ids[i]],
                'confidence': confidences[i],
                'box': [x, y, w, h]
            })

    return detections, img_rgb

# Hàm vẽ kết quả phát hiện
def draw_detections(img, detections):
    for det in detections:
        x, y, w, h = det['box']
        label = det['class_name']
        confidence = det['confidence']
        color = (255, 0, 0)  # Màu xanh dương cho bounding box

        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
        cv2.putText(img, f"{label} {confidence:.2f}", (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    plt.figure(figsize=(12, 8))
    plt.imshow(img)
    plt.axis('off')
    plt.show()

# Hàm lấy tọa độ các góc
def get_corner_points(detections):
    corners = {}
    for det in detections:
        class_name = det['class_name']
        box = det['box']
        x, y, w, h = box
        center_x = x + w / 2
        center_y = y + h / 2
        corners[class_name] = (center_x, center_y)
    return corners

# Hàm biến đổi phối cảnh
def four_point_transform(image, corners):
    required_keys = ['top-left', 'top-right', 'bottom-right', 'bottom-left']
    for key in required_keys:
        if key not in corners:
            raise ValueError(f"Thiếu góc: {key}")

    pts1 = np.float32([
        corners['top-left'],
        corners['top-right'],
        corners['bottom-right'],
        corners['bottom-left']
    ])

    widthA = np.linalg.norm(np.array(corners['bottom-right']) - np.array(corners['bottom-left']))
    widthB = np.linalg.norm(np.array(corners['top-right']) - np.array(corners['top-left']))
    maxWidth = max(int(widthA), int(widthB))

    heightA = np.linalg.norm(np.array(corners['top-right']) - np.array(corners['bottom-right']))
    heightB = np.linalg.norm(np.array(corners['top-left']) - np.array(corners['bottom-left']))
    maxHeight = max(int(heightA), int(heightB))

    pts2 = np.float32([
        [0, 0],
        [maxWidth - 1, 0],
        [maxWidth - 1, maxHeight - 1],
        [0, maxHeight - 1]
    ])

    matrix = cv2.getPerspectiveTransform(pts1, pts2)
    warped = cv2.warpPerspective(image, matrix, (maxWidth, maxHeight))

    return warped

# Hàm lưu và hiển thị ảnh (cập nhật để update thay vì insert mới)
def save_and_display(warped_image):
    input_image_name = uploaded_file_widget.value[0]['name']  
    
    output_image_path = os.path.join(IMG_OUTPUT_DIR,
                                      f"cropped_{os.path.basename(input_image_name)}")

    plt.figure(figsize=(12, 8))
    plt.imshow(cv2.cvtColor(warped_image, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title('Căn cước công dân đã cắt và chỉnh sửa')
    plt.show()

    # Kiểm tra xem ảnh đã tồn tại trong cơ sở dữ liệu chưa trước khi lưu
    cursor.execute('SELECT COUNT(*) FROM gocsau WHERE input_name=?', (input_image_name.split('/')[-1],))
    
    if cursor.fetchone()[0] == 0:  
        # Nếu không có bản ghi nào thì mới chèn vào cơ sở dữ liệu
        # Lưu ảnh đã cắt
        cv2.imwrite(output_image_path, warped_image)

        # Lưu đường dẫn vào cơ sở dữ liệu
        cursor.execute('INSERT INTO gocsau (input_name, output_path) VALUES (?, ?)', 
                       (input_image_name.split('/')[-1], output_image_path))
        
        print(f"Đã lưu ảnh mới vào cơ sở dữ liệu: {output_image_path}")
    
    else:
        # Nếu đã tồn tại thì có thể cập nhật đường dẫn mới hoặc thông báo
        cursor.execute('UPDATE gocsau SET output_path=? WHERE input_name=?', 
                       (output_image_path , input_image_name.split('/')[-1]))
        
        # Lưu ảnh đã cắt nếu cần thiết (có thể bỏ qua nếu không muốn ghi đè)
        cv2.imwrite(output_image_path , warped_image)  
        
        print(f"Đã cập nhật đường dẫn ảnh trong cơ sở dữ liệu: {output_image_path}")

def clear_and_display(warped_image):
   save_and_display(warped_image)

# Xóa dữ liệu cũ trước khi bắt đầu (có thể điều chỉnh tham số clear_data nếu cần)
clear_old_images()

# Tạo widget để chọn tệp ảnh
uploaded_file_widget = widgets.FileUpload(
    accept='image/*',
    multiple=False,
)

display(uploaded_file_widget)

def process_image(change):
    if uploaded_file_widget.value:
        # Lấy thông tin tệp được tải lên từ danh sách đầu tiên
        uploaded_file_info = uploaded_file_widget.value[0]  
        
        # Lưu tệp vào một vị trí tạm thời để xử lý
        with tempfile.NamedTemporaryFile(delete=False) as temp_file:
            temp_file.write(uploaded_file_info['content'])
            temp_file_path = temp_file.name
        
        # Thực hiện phát hiện và cắt góc
        detections , img_rgb= detect_objects(temp_file_path)
        
        draw_detections(img_rgb.copy(), detections)
        
        corners= get_corner_points(detections)

        try:
            warped_image= four_point_transform(cv2.cvtColor(img_rgb.copy(), cv2.COLOR_RGB2BGR), corners)
            clear_and_display(warped_image) 
            conn.commit()  
            
            print("Đã hoàn thành quá trình xử lý.")
            
        except ValueError as e:
            print(e)

uploaded_file_widget.observe(process_image) 

# Đóng kết nối cơ sở dữ liệu khi hoàn tất (nên thực hiện sau khi chắc chắn không còn thao tác nào nữa).
def close_connection():
   conn.close()


In [ ]:

# Đường dẫn tới các tệp mô hình
model_dir = '../models'
config_path = os.path.join(model_dir, 'yolov4-obj.cfg')
weights_path = os.path.join(model_dir, 'yolov4-obj_final.weights')
names_path = os.path.join(model_dir, 'obj2.names')

# Kết nối đến cơ sở dữ liệu
DB_PATH = '../data/database.db'
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

# Tạo bảng detecttrc nếu chưa tồn tại
cursor.execute('''CREATE TABLE IF NOT EXISTS detecttrc (input_name TEXT, output_path TEXT)''')
conn.commit()

# Xóa dữ liệu cũ trong bảng detecttrc
def clear_old_data():
    cursor.execute('DELETE FROM detecttrc')
    conn.commit()
    print("Đã xóa dữ liệu cũ trong bảng detecttrc.")

# Đọc các lớp đối tượng
classes = []
with open(names_path, 'r') as f:
    classes = [line.strip() for line in f.readlines()]
print("Classes:", classes)

# Load mô hình
net = cv2.dnn.readNet(weights_path, config_path)
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
conf_threshold = 0.5
nms_threshold = 0.4

# Hàm phát hiện đối tượng
def detect_objects(img_path):
    img = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Chuyển đổi sang RGB để hiển thị
    height, width, channels = img.shape

    blob = cv2.dnn.blobFromImage(img, 1/255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    outputs = net.forward(output_layers)

    class_ids, confidences, boxes = [], [], []

    for output in outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > conf_threshold:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)

    detections = []
    if len(indexes) > 0:
        for i in indexes.flatten():
            x, y, w, h = boxes[i]
            detections.append({
                'class_id': class_ids[i],
                'class_name': classes[class_ids[i]],
                'confidence': confidences[i],
                'box': [x, y, w, h]
            })

    return detections, img_rgb

# Hàm vẽ bounding box
def draw_bounding_boxes(img, detections):
    for det in detections:
        x, y, w, h = det['box']
        label = det['class_name']
        confidence = det['confidence']
        color = (0, 255, 0)  # Màu xanh lá cây cho bounding box

        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
        cv2.putText(img, f"{label}: {confidence:.2f}", (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    plt.figure(figsize=(12, 8))
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title("Detected Fields with Bounding Boxes")
    plt.axis('off')
    plt.show()

def crop_and_sort_bounding_boxes_v2(img_original, detections):
    output_dir = '../data/img/croptrc/'
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Nhóm các bounding box theo class_name
    grouped_boxes = {}
    for det in detections:
        class_name = det['class_name']
        if class_name not in grouped_boxes:
            grouped_boxes[class_name] = []
        grouped_boxes[class_name].append(det['box'])

    # Hàm sắp xếp bounding box
    def sort_boxes_v2(boxes):
        boxes.sort(key=lambda b: b[1])  # Sắp xếp theo tọa độ y

        grouped_rows = []
        current_row = []

        for box in boxes:
            if not current_row:
                current_row.append(box)
            else:
                _, y1, _, _ = current_row[-1]
                _, y2, _, _ = box
                if abs(y1 - y2) <= 20:  # Cùng hàng
                    current_row.append(box)
                elif abs(y1 - y2) > 30:  # Xuống hàng mới
                    grouped_rows.append(sorted(current_row, key=lambda b: b[0]))
                    current_row = [box]

        if current_row:
            grouped_rows.append(sorted(current_row, key=lambda b: b[0]))

        sorted_boxes = [box for row in grouped_rows for box in row]
        return sorted_boxes

    # Xử lý từng class_name
    for class_name, boxes in grouped_boxes.items():
        print(f"\nProcessing class: {class_name}")
        sorted_boxes = sort_boxes_v2(boxes)

        for i, (x,y,w,h) in enumerate(sorted_boxes):
            # Cắt ảnh
            cropped_img = img_original[y:y+h, x:x+w]
            
            # Nếu là trường img thì lưu ảnh gốc mà không thay đổi gì và chuyển đổi sang RGB
            if class_name == "img":
                output_img_rgb = cv2.cvtColor(cropped_img, cv2.COLOR_BGR2RGB)
            else:
                # Chuyển đổi sang ảnh nhị phân cho các trường khác ngoại trừ "img"
                gray_img = cv2.cvtColor(cropped_img, cv2.COLOR_BGR2GRAY)
                _, output_img_rgb = cv2.threshold(gray_img, 127, 255, cv2.THRESH_BINARY)

            # Đổi tên file theo thứ tự và lưu vào thư mục đầu ra
            output_path = os.path.join(output_dir, f"{class_name}_{i + 1}.jpg")
            cv2.imwrite(output_path, output_img_rgb)

            # Lưu thông tin vào cơ sở dữ liệu
            cursor.execute('INSERT INTO detecttrc (input_name, output_path) VALUES (?, ?)', 
                           (f"{class_name}_{i + 1}.jpg", output_path))

    conn.commit()
    
# Xóa dữ liệu cũ trước khi bắt đầu
clear_old_data()

# Lấy đường dẫn ảnh đầu vào từ bảng goctrc
cursor.execute('SELECT output_path FROM goctrc')
rows = cursor.fetchall()
if rows:
    input_image_path = rows[0][0]  # Lấy đường dẫn đầu tiên từ bảng goctrc

# Phát hiện đối tượng
detections, img_rgb = detect_objects(input_image_path)

# Vẽ các bounding box trên ảnh
draw_bounding_boxes(img_rgb.copy(), detections)

# Cắt và lưu từng bounding box thành ảnh riêng biệt với ảnh gốc cho trường img
crop_and_sort_bounding_boxes_v2(img_rgb.copy(), detections)

# Đóng kết nối cơ sở dữ liệu
conn.close()


In [ ]:

# Đường dẫn tới các tệp mô hình
model_dir = '../models'
config_path = os.path.join(model_dir, 'yolo-obj4.cfg')
weights_path = os.path.join(model_dir, 'yolo-obj4_final.weights')
names_path = os.path.join(model_dir, 'obj4.names')

# Kết nối đến cơ sở dữ liệu
DB_PATH = '../data/database.db'
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

# Tạo bảng detectsau nếu chưa tồn tại
cursor.execute('''CREATE TABLE IF NOT EXISTS detectsau (input_name TEXT, output_path TEXT)''')
conn.commit()

# Xóa dữ liệu cũ trong bảng detectsau
def clear_old_data():
    cursor.execute('DELETE FROM detectsau')
    conn.commit()
    print("Đã xóa dữ liệu cũ trong bảng detectsau.")

# Đọc các lớp đối tượng
classes = []
with open(names_path, 'r') as f:
    classes = [line.strip() for line in f.readlines()]
print("Classes:", classes)

# Load mô hình
net = cv2.dnn.readNet(weights_path, config_path)
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
conf_threshold = 0.5
nms_threshold = 0.4

# Hàm phát hiện đối tượng
def detect_objects(img_path):
    img = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Chuyển đổi sang RGB để hiển thị
    height, width, channels = img.shape

    blob = cv2.dnn.blobFromImage(img, 1/255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    outputs = net.forward(output_layers)

    class_ids, confidences, boxes = [], [], []

    for output in outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > conf_threshold:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    indexes = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)

    detections = []
    if len(indexes) > 0:
        for i in indexes.flatten():
            x, y, w, h = boxes[i]
            detections.append({
                'class_id': class_ids[i],
                'class_name': classes[class_ids[i]],
                'confidence': confidences[i],
                'box': [x, y, w, h]
            })

    return detections, img_rgb

# Hàm vẽ bounding box
def draw_bounding_boxes(img, detections):
    for det in detections:
        x, y, w, h = det['box']
        label = det['class_name']
        confidence = det['confidence']
        color = (0, 255, 0)  # Màu xanh lá cây cho bounding box

        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
        cv2.putText(img, f"{label}: {confidence:.2f}", (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    plt.figure(figsize=(12, 8))
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title("Detected Fields with Bounding Boxes")
    plt.axis('off')
    plt.show()

def crop_and_sort_bounding_boxes_v2(img_original, detections):
    output_dir = '../data/img/cropsau/'
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Nhóm các bounding box theo class_name
    grouped_boxes = {}
    for det in detections:
        class_name = det['class_name']
        if class_name not in grouped_boxes:
            grouped_boxes[class_name] = []
        grouped_boxes[class_name].append(det['box'])

    # Hàm sắp xếp bounding box
    def sort_boxes_v2(boxes):
        boxes.sort(key=lambda b: b[1])  # Sắp xếp theo tọa độ y

        grouped_rows = []
        current_row = []

        for box in boxes:
            if not current_row:
                current_row.append(box)
            else:
                _, y1, _, _ = current_row[-1]
                _, y2, _, _ = box
                if abs(y1 - y2) <= 20:  # Cùng hàng
                    current_row.append(box)
                elif abs(y1 - y2) > 30:  # Xuống hàng mới
                    grouped_rows.append(sorted(current_row, key=lambda b: b[0]))
                    current_row = [box]

        if current_row:
            grouped_rows.append(sorted(current_row, key=lambda b: b[0]))

        sorted_boxes = [box for row in grouped_rows for box in row]
        return sorted_boxes

    # Xử lý từng class_name
    for class_name, boxes in grouped_boxes.items():
        print(f"\nProcessing class: {class_name}")
        sorted_boxes = sort_boxes_v2(boxes)

        for i, (x,y,w,h) in enumerate(sorted_boxes):
            # Cắt ảnh
            cropped_img = img_original[y:y+h, x:x+w]
            
            # Nếu là trường img thì lưu ảnh gốc mà không thay đổi gì và chuyển đổi sang RGB
            if class_name == "img":
                output_img_rgb = cv2.cvtColor(cropped_img, cv2.COLOR_BGR2RGB)
            else:
                # Chuyển đổi sang ảnh nhị phân cho các trường khác ngoại trừ "img"
                gray_img = cv2.cvtColor(cropped_img, cv2.COLOR_BGR2GRAY)
                _, output_img_rgb = cv2.threshold(gray_img, 127, 255, cv2.THRESH_BINARY)

            # Đổi tên file theo thứ tự và lưu vào thư mục đầu ra
            output_path = os.path.join(output_dir, f"{class_name}_{i + 1}.jpg")
            cv2.imwrite(output_path, output_img_rgb)

            # Lưu thông tin vào cơ sở dữ liệu
            cursor.execute('INSERT INTO detectsau (input_name, output_path) VALUES (?, ?)', 
                           (f"{class_name}_{i + 1}.jpg", output_path))

    conn.commit()
    
# Xóa dữ liệu cũ trước khi bắt đầu
clear_old_data()

# Lấy đường dẫn ảnh đầu vào từ bảng gocsau
cursor.execute('SELECT output_path FROM gocsau')
rows = cursor.fetchall()
if rows:
    input_image_path = rows[0][0]  # Lấy đường dẫn đầu tiên từ bảng gocsau

# Phát hiện đối tượng
detections, img_rgb = detect_objects(input_image_path)

# Vẽ các bounding box trên ảnh
draw_bounding_boxes(img_rgb.copy(), detections)

# Cắt và lưu từng bounding box thành ảnh riêng biệt với ảnh gốc cho trường img
crop_and_sort_bounding_boxes_v2(img_rgb.copy(), detections)

# Đóng kết nối cơ sở dữ liệu
conn.close()


In [ ]:

import pytesseract
from collections import defaultdict
import re
import json

# Cấu hình đường dẫn Tesseract OCR
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

# Kết nối đến cơ sở dữ liệu
DB_PATH = '../data/database.db'
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

# Tạo bảng detailtrc nếu chưa tồn tại
cursor.execute('''CREATE TABLE IF NOT EXISTS detailtrc (input_name TEXT, output_path TEXT)''')

# Hàm loại bỏ các ký tự đặc biệt như \", dấu phẩy, dấu cách thừa
def clean_text(text):
    cleaned_text = re.sub(r'["\s;,\n\r\t]+', ' ', text)  # Loại bỏ \" , ; dấu cách thừa
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()  # Thay nhiều khoảng trắng thành một
    return cleaned_text

# Hàm xóa dữ liệu cũ trong bảng detailtrc
def clear_old_data():
    cursor.execute('DELETE FROM detailtrc')
    conn.commit()
    print("Đã xóa dữ liệu cũ trong bảng detailtrc.")

# Hàm chính để xử lý OCR và gộp kết quả
def process_ocr_and_merge(input_dir):
    # Danh sách lưu kết quả tạm thời
    extracted_data = defaultdict(list)  # Dùng để gộp các trường cùng loại

    # Sử dụng OCR để trích xuất văn bản từ ảnh trong thư mục
    img_path = input_dir  # input_dir là một đường dẫn đến một tệp chứ không phải thư mục
    img_file = os.path.basename(img_path)
    
    img = cv2.imread(img_path)

    # Kiểm tra xem tên file có phải là img hay không, nếu có thì chỉ lưu đường dẫn ảnh
    field_name = os.path.splitext(img_file)[0].split("_")[0]  # Lấy tên trường từ tên file

    if field_name == "img":
        # Nếu là trường img, chỉ cần lưu đường dẫn ảnh vào cơ sở dữ liệu mà không thực hiện OCR
        cursor.execute('INSERT INTO detailtrc (input_name, output_path) VALUES (?, ?)', 
                       (field_name, img_path))
        print(f"Field: {field_name} - Image path saved without OCR: {img_path}")
        return  # Kết thúc hàm nếu là trường img
    
    # Sử dụng OCR để trích xuất văn bản cho các trường khác
    text = pytesseract.image_to_string(img, lang="vie").strip()

    # Nếu văn bản là bytes, chuyển đổi thành chuỗi str (nếu cần)
    if isinstance(text, bytes):
        text = text.decode('utf-8')

    # Làm sạch văn bản (loại bỏ các ký tự đặc biệt như \", dấu phẩy, dấu cách thừa)
    cleaned_text = clean_text(text)

    # Lưu kết quả vào danh sách
    extracted_data[field_name].append(cleaned_text)

    # Hiển thị ảnh và văn bản đã trích xuất (đã làm sạch)
    print(f"Field: {field_name}")
    print(f"Extracted Text: {cleaned_text}")
    print("-" * 40)

    # Lưu thông tin vào cơ sở dữ liệu detailtrc
    for field, values in extracted_data.items():
        for value in values:
            cursor.execute('INSERT INTO detailtrc (input_name, output_path) VALUES (?, ?)', 
                           (field, value))

    conn.commit()

# Xóa dữ liệu cũ trước khi bắt đầu
clear_old_data()

# Lấy đường dẫn ảnh từ cơ sở dữ liệu detecttrc
cursor.execute('SELECT output_path FROM detecttrc')
rows = cursor.fetchall()

# Duyệt qua từng đường dẫn ảnh lấy được từ cơ sở dữ liệu và gọi hàm xử lý OCR
for row in rows:
    input_dir = row[0]  # input_dir là đường dẫn đến tệp ảnh

    print(f"Processing image: {input_dir}")
    
    process_ocr_and_merge(input_dir)

# Gộp các trường giống nhau và lưu vào file JSON
final_data = defaultdict(list)

# Lấy tất cả dữ liệu từ bảng detailtrc để gộp lại
cursor.execute('SELECT input_name, output_path FROM detailtrc')
rows = cursor.fetchall()

for input_name, output_path in rows:
    if input_name.startswith("home"):  # Gộp các trường home_1, home_2, home_3 thành home
        final_data['home'].append(output_path)
    else:
        final_data[input_name].append(output_path)

# Chuyển đổi danh sách thành chuỗi cho các trường gộp lại
for key in final_data.keys():
    final_data[key] = " ".join(final_data[key])

# Lưu kết quả vào file JSON
output_json_path = '../data/datatrc.json'
with open(output_json_path, 'w', encoding='utf-8') as json_file:
    json.dump(final_data, json_file, ensure_ascii=False, indent=4)

print(f"\nData saved to {output_json_path}")

# Đóng kết nối cơ sở dữ liệu
conn.close()


In [ ]:

# Cấu hình đường dẫn Tesseract OCR
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

# Kết nối đến cơ sở dữ liệu
DB_PATH = '../data/database.db'
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

# Tạo bảng detailsau nếu chưa tồn tại
cursor.execute('''CREATE TABLE IF NOT EXISTS detailsau (input_name TEXT, output_path TEXT)''')

# Hàm loại bỏ các ký tự đặc biệt như \", dấu phẩy, dấu cách thừa
def clean_text(text):
    cleaned_text = re.sub(r'["\s;,\n\r\t]+', ' ', text)  # Loại bỏ \" , ; dấu cách thừa
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()  # Thay nhiều khoảng trắng thành một
    return cleaned_text

# Hàm xóa dữ liệu cũ trong bảng detailsau
def clear_old_data():
    cursor.execute('DELETE FROM detailsau')
    conn.commit()
    print("Đã xóa dữ liệu cũ trong bảng detailsau.")

# Hàm chính để xử lý OCR và gộp kết quả
def process_ocr_and_merge(input_dir):
    # Danh sách lưu kết quả tạm thời
    extracted_data = defaultdict(list)  # Dùng để gộp các trường cùng loại

    # Sử dụng OCR để trích xuất văn bản từ ảnh trong thư mục
    img_path = input_dir  # input_dir là một đường dẫn đến một tệp chứ không phải thư mục
    img_file = os.path.basename(img_path)
    
    img = cv2.imread(img_path)

    # Kiểm tra xem tên file có phải là img hay không, nếu có thì chỉ lưu đường dẫn ảnh
    field_name = os.path.splitext(img_file)[0].split("_")[0]  # Lấy tên trường từ tên file

    if field_name == "img":
        # Nếu là trường img, chỉ cần lưu đường dẫn ảnh vào cơ sở dữ liệu mà không thực hiện OCR
        cursor.execute('INSERT INTO detailsau (input_name, output_path) VALUES (?, ?)', 
                       (field_name, img_path))
        print(f"Field: {field_name} - Image path saved without OCR: {img_path}")
        return  # Kết thúc hàm nếu là trường img
    
    # Sử dụng OCR để trích xuất văn bản cho các trường khác
    text = pytesseract.image_to_string(img, lang="vie").strip()

    # Nếu văn bản là bytes, chuyển đổi thành chuỗi str (nếu cần)
    if isinstance(text, bytes):
        text = text.decode('utf-8')

    # Làm sạch văn bản (loại bỏ các ký tự đặc biệt như \", dấu phẩy, dấu cách thừa)
    cleaned_text = clean_text(text)

    # Lưu kết quả vào danh sách
    extracted_data[field_name].append(cleaned_text)

    # Hiển thị ảnh và văn bản đã trích xuất (đã làm sạch)
    print(f"Field: {field_name}")
    print(f"Extracted Text: {cleaned_text}")
    print("-" * 40)

    # Lưu thông tin vào cơ sở dữ liệu detailsau
    for field, values in extracted_data.items():
        for value in values:
            cursor.execute('INSERT INTO detailsau (input_name, output_path) VALUES (?, ?)', 
                           (field, value))

    conn.commit()

# Xóa dữ liệu cũ trước khi bắt đầu
clear_old_data()

# Lấy đường dẫn ảnh từ cơ sở dữ liệu detectsau
cursor.execute('SELECT output_path FROM detectsau')
rows = cursor.fetchall()

# Duyệt qua từng đường dẫn ảnh lấy được từ cơ sở dữ liệu và gọi hàm xử lý OCR
for row in rows:
    input_dir = row[0]  # input_dir là đường dẫn đến tệp ảnh

    print(f"Processing image: {input_dir}")
    
    process_ocr_and_merge(input_dir)

# Gộp các trường giống nhau và lưu vào file JSON
final_data = defaultdict(list)

# Lấy tất cả dữ liệu từ bảng detailsau để gộp lại
cursor.execute('SELECT input_name, output_path FROM detailsau')
rows = cursor.fetchall()

for input_name, output_path in rows:
    if input_name.startswith("ddnd"):  
        final_data['ddnd'].append(output_path)
    else:
        final_data[input_name].append(output_path)

# Chuyển đổi danh sách thành chuỗi cho các trường gộp lại
for key in final_data.keys():
    final_data[key] = " ".join(final_data[key])

# Lưu kết quả vào file JSON
output_json_path = '../data/datasau.json'
with open(output_json_path, 'w', encoding='utf-8') as json_file:
    json.dump(final_data, json_file, ensure_ascii=False, indent=4)

print(f"\nData saved to {output_json_path}")

# Đóng kết nối cơ sở dữ liệu
conn.close()


In [ ]:
from transformers import pipeline

# Kết nối đến cơ sở dữ liệu
DB_PATH = '../data/database.db'
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

# Tạo bảng full nếu chưa tồn tại
cursor.execute('''CREATE TABLE IF NOT EXISTS full (field TEXT, content TEXT)''')
conn.commit()

# Hàm xóa dữ liệu cũ trong bảng full
def clear_old_data():
    cursor.execute('DELETE FROM full')
    conn.commit()
    print("Đã xóa dữ liệu cũ trong bảng full.")

# Hàm làm sạch văn bản bằng Regular Expressions
def clean_text(text):
    # Loại bỏ các ký tự không mong muốn
    cleaned_text = re.sub(r'[\"_;(-]', '', text)  # Loại bỏ dấu ngoặc kép, gạch dưới và dấu chấm phẩy
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text).strip()  # Thay nhiều khoảng trắng thành một
    return cleaned_text

# Hàm chỉnh sửa chính tả và lưu vào DB
def correct_spelling_and_save():
    # Khởi tạo pipeline cho việc chỉnh sửa chính tả
    corrector = pipeline("text2text-generation", model="bmd1905/vietnamese-correction-v2")

    # Lấy dữ liệu từ bảng detailtrc
    cursor.execute('SELECT input_name, output_path FROM detailtrc')
    detailtrc_rows = cursor.fetchall()

    # Lấy dữ liệu từ bảng detailsau
    cursor.execute('SELECT input_name, output_path FROM detailsau')
    detailsau_rows = cursor.fetchall()

    # Lưu kết quả vào bảng full
    for input_name, output_path in detailtrc_rows + detailsau_rows:  # Kết hợp cả hai bảng
        corrected_text = corrector(output_path)[0]['generated_text']
        cleaned_text = clean_text(corrected_text)  # Làm sạch văn bản đã chỉnh sửa
        cursor.execute('INSERT INTO full (field, content) VALUES (?, ?)', (input_name, cleaned_text))

    conn.commit()
    print("Dữ liệu đã được lưu vào bảng full.")

# Hàm xuất dữ liệu ra file JSON theo định dạng yêu cầu
def export_to_json():
    cursor.execute('SELECT field, content FROM full')
    rows = cursor.fetchall()

    # Tạo một từ điển để lưu trữ kết quả theo định dạng yêu cầu
    json_output = {}

    for field, content in rows:
        # Cập nhật giá trị cho từng trường trong json_output
        if field in json_output:
            # Nếu trường đã tồn tại, có thể gộp hoặc xử lý theo cách nào đó nếu cần thiết.
            json_output[field] += f", {content}"  # Gộp nội dung nếu cần thiết (có thể điều chỉnh)
        else:
            json_output[field] = content

    # Xuất ra file JSON
    with open('../data/output.json', 'w', encoding='utf-8') as json_file:
        json.dump(json_output, json_file, ensure_ascii=False, indent=4)
    
    print("Dữ liệu đã được xuất ra file output.json.")

# Xóa dữ liệu cũ trước khi bắt đầu
clear_old_data()

# Chạy hàm chỉnh sửa chính tả và lưu kết quả
correct_spelling_and_save()

# Xuất dữ liệu ra file JSON
export_to_json()

# Đóng kết nối cơ sở dữ liệu
conn.close()
